In [1]:
# external imports
import music21 as m21
import ast
import os

In [2]:
# reload cell
from src.midi import *
from src.train import *
import sys, importlib
importlib.reload(sys.modules['src.midi'])
importlib.reload(sys.modules['src.train'])

<module 'src.train' from 'D:\\Documents\\GitHub\\Pyotr\\src\\train.py'>

# Encoding and Read-in

In [3]:
# Read in all of the midi files
path = './data/sample'
mdl = gen_md_from_path(path, by_measure=False, verbose=False)
mdm = gen_md_from_path(path, by_measure=True, verbose=False)

In [4]:
# Encode them (as entire piece)
me = MidiEncoder()
mdl_enc = {}
for piece in mdl:
    mdl_enc[piece] = me.Encode(mdl[piece].flat, 'pitch_position_duration_strings')

In [5]:
# Encode them (by measure)
me = MidiEncoder()
mdm_enc = {}
for piece in mdm:
    mdm_enc[piece] = {}
    for i, m in enumerate(mdm[piece]):
        mdm_enc[piece][i] = me.Encode(m, 'pitch_position_duration_strings')

In [6]:
# Vocab Sizes
un = []; count = 0;
for piece in mdl_enc:
    for token in mdl_enc[piece]:
        count += 1
        if token not in un:
            un.append(token)
print(f"{len(un)} / {count}")

12258 / 14921


In [7]:
# Vocab Sizes
un = []; count = 0;
for piece in mdm_enc:
    for meas in mdm_enc[piece]:
        for token in mdm_enc[piece][meas]:
            count += 1
            if token not in un:
                un.append(token)
print(f"{len(un)} / {count}")

2762 / 14921


# Pre-processing

In [6]:
tsg = TrainingSetBuilder(mdm_enc, by_measure=True)
tsg.Build('next_note')

In [7]:
tsg.mdict_enc_f

{'wtc1011': ['C4:0.0:2.0',
  'E4:0.25:1.75',
  'G4:0.5:0.25',
  'C5:0.75:0.25',
  'E5:1.0:0.25',
  'G4:1.25:0.25',
  'C5:1.5:0.25',
  'E5:1.75:0.25',
  'C4:2.0:2.0',
  'E4:2.25:1.75',
  'G4:2.5:0.25',
  'C5:2.75:0.25',
  'E5:3.0:0.25',
  'G4:3.25:0.25',
  'C5:3.5:0.25',
  'E5:3.75:0.25',
  'C4:0.0:2.0',
  'D4:0.25:1.75',
  'A4:0.5:0.25',
  'D5:0.75:0.25',
  'F5:1.0:0.25',
  'A4:1.25:0.25',
  'D5:1.5:0.25',
  'F5:1.75:0.25',
  'C4:2.0:2.0',
  'D4:2.25:1.75',
  'A4:2.5:0.25',
  'D5:2.75:0.25',
  'F5:3.0:0.25',
  'A4:3.25:0.25',
  'D5:3.5:0.25',
  'F5:3.75:0.25',
  'B3:0.0:2.0',
  'D4:0.25:1.75',
  'G4:0.5:0.25',
  'D5:0.75:0.25',
  'F5:1.0:0.25',
  'G4:1.25:0.25',
  'D5:1.5:0.25',
  'F5:1.75:0.25',
  'B3:2.0:2.0',
  'D4:2.25:1.75',
  'G4:2.5:0.25',
  'D5:2.75:0.25',
  'F5:3.0:0.25',
  'G4:3.25:0.25',
  'D5:3.5:0.25',
  'F5:3.75:0.25',
  'C4:0.0:2.0',
  'E4:0.25:1.75',
  'G4:0.5:0.25',
  'C5:0.75:0.25',
  'E5:1.0:0.25',
  'G4:1.25:0.25',
  'C5:1.5:0.25',
  'E5:1.75:0.25',
  'C4:2.0:2.0',


# Modeling